<br><br><br><br><br>

# Accelerating Python

<br><br><br><br><br>

<br>

<p style="font-size: 1.25em">Apart from clever algorithms, the speed of a program is determined by how laden it is with work unrelated to its main task.</p>

<center><img src="img/swallows-coconut.jpg" width="50%"></center>

<p style="font-size: 1.25em">Python is slower than C because of dynamic type checking, garbage collection, everything-is-a-hashtable, pointer chasing, string equality checks...</p>

<br>

<p style="font-size: 1.25em">Compare to Java, which has garbage collection but not dynamic type checking, and C, which has neither (on a variety of benchmark algorithms).</p>

<img src="img/benchmark-games.png" width="100%">

<br><br>

<p style="font-size: 1.25em">If you care about speed <i>more than</i> ease of development, use C or C++ (or Rust!).</p>

<br>

<p style="font-size: 1.25em">If you want to develop in Python for all of its conveniences—dynamic type checking, garbage collection, everything-is-a-hashtable, interactive development and debugging—and need to speed up a critical section, there are ways to do it.</p>

<br>

<p style="font-size: 1.25em">But you have to give up on some Python friendliness <i>in that section.</i></p>

<br><br>

In [ ]:
# Some wacky things you can do with Python:

class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

point = Point(1, 2)

